<a href="https://colab.research.google.com/github/azamatomu/msc_thesis/blob/master/tfrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 3.6MB 9.1MB/s 


In [3]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data

In [10]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))


Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [5]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [14]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(512, activation="relu"),
      tf.keras.layers.Dense(128, activation="relu"),
      tf.keras.layers.Dense(32, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
    ])
    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])
    rating_predictions = self.ratings(tf.concat([user_embeddings, movie_embeddings], axis=1))
    
    return self.task(labels=features["user_rating"], predictions=rating_predictions)

Define the two models and the retrieval task.

In [15]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

task = tfrs.tasks.Ranking(
                loss = tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.RootMeanSquaredError()]
)



### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [17]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adam(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=10)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/10
25/25 [==============================] - 4s 137ms/step - root_mean_squared_error: 21124.0703 - loss: 419007611.6780 - regularization_loss: 0.0000e+00 - total_loss: 419007611.6780
Epoch 2/10
25/25 [==============================] - 3s 133ms/step - root_mean_squared_error: 5.8499 - loss: 34.2836 - regularization_loss: 0.0000e+00 - total_loss: 34.2836
Epoch 3/10
25/25 [==============================] - 3s 133ms/step - root_mean_squared_error: 5.9351 - loss: 35.2383 - regularization_loss: 0.0000e+00 - total_loss: 35.2383
Epoch 4/10
25/25 [==============================] - 3s 132ms/step - root_mean_squared_error: 5.9371 - loss: 35.2579 - regularization_loss: 0.0000e+00 - total_loss: 35.2579
Epoch 5/10
 1/25 [>.............................] - ETA: 4s - root_mean_squared_error: 5.9463 - loss: 35.3585 - regularization_loss: 0.0000e+00 - total_loss: 35.3585

KeyboardInterrupt: ignored